In [ ]:
# Copyright 2022 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions anda
# limitations under the License.
# ==============================================================================

<img src="http://developer.download.nvidia.com/compute/machine-learning/frameworks/nvidia_logo.png" style="width: 90px; float: right;">

# Training with pretrained embeddings

## Overview

In this use case we will consider how we might train with pretrained embeddings.

Pretrained embeddings can allow our model to include information from additional modalities (for instance, we might want to grab CNN descriptors of product images). They can also come from other models that we train on our data. For example, we might train a word2vec model on the sequence of purchased items by a customer and want to include this information in our retrieval or ranking model.

The use cases are many, but this particular example will focus on the technical aspects of working with pretrained embeddings.

We will use the MovieLens 100k dataset and emulate a scenario where we would have a pretrained embedding for each of the movies in the train dataset.

### Learning objectives

- Training with pretrained embeddings
- Understanding [the Schema file](https://github.com/NVIDIA-Merlin/core/blob/main/merlin/schema/schema.py)

## Downloading and preparing the dataset

In [1]:
import merlin.models.tf as mm
from merlin.datasets.entertainment import get_movielens
from merlin.schema.tags import Tags
import tensorflow as tf
from merlin.models.tf.prediction_tasks.classification import BinaryClassificationTask
from merlin.models.tf.blocks import *

import numpy as np

2022-06-20 21:53:41.425846: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:952] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-20 21:53:41.426187: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:952] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-20 21:53:41.426330: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:952] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-20 21:53:41.447601: I tensorflow/core/platform/cpu_feature_guard.cc:152] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate 

In [2]:
train, valid = get_movielens(variant="ml-100k")

/usr/local/lib/python3.8/dist-packages/cudf/core/dataframe.py:1292: UserWarning: The deep parameter is ignored and is only included for pandas compatibility.
  warnings.warn(


In [3]:
target_column = train.schema.select_by_tag(Tags.TARGET).column_names[1]
target_column

'rating_binary'

In [4]:
train.schema

,name,tags,dtype,is_list,is_ragged,properties.num_buckets,properties.freq_threshold,properties.max_size,properties.start_index,properties.cat_path,properties.embedding_sizes.cardinality,properties.embedding_sizes.dimension,properties.domain.min,properties.domain.max
0,movieId,"(Tags.ITEM, Tags.CATEGORICAL, Tags.ITEM_ID)",int32,False,False,NaN,0.0,0.0,0.0,.//categories/unique.movieId.parquet,1681.0,102.0,0.0,1681.0
1,userId,"(Tags.CATEGORICAL, Tags.USER, Tags.USER_ID)",int32,False,False,NaN,0.0,0.0,0.0,.//categories/unique.userId.parquet,944.0,74.0,0.0,944.0
2,genres,"(Tags.CATEGORICAL, Tags.ITEM)",int32,False,False,NaN,0.0,0.0,0.0,.//categories/unique.genres.parquet,217.0,33.0,0.0,217.0
3,TE_movieId_rating,(Tags.CONTINUOUS),float64,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,userId_count,(Tags.CONTINUOUS),float32,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,gender,"(Tags.CATEGORICAL, Tags.USER)",int32,False,False,NaN,0.0,0.0,0.0,.//categories/unique.gender.parquet,3.0,16.0,0.0,3.0
6,zip_code,"(Tags.CATEGORICAL, Tags.USER)",int32,False,False,NaN,0.0,0.0,0.0,.//categories/unique.zip_code.parquet,796.0,67.0,0.0,796.0
7,rating,"(Tags.TARGET, Tags.REGRESSION)",int64,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,rating_binary,"(Tags.TARGET, Tags.BINARY_CLASSIFICATION)",int32,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,age,"(Tags.CATEGORICAL, Tags.USER)",int32,False,False,NaN,0.0,0.0,0.0,.//categories/unique.age.parquet,9.0,16.0,0.0,9.0


From the schema, we can tell that there are 1681 known movie ids in the dataset.

Let's us read the file storing the mapping of moveIds to movies and take a closer look at the situation.

In [5]:
import cudf

movieIds = cudf.read_parquet(train.schema.column_schemas['movieId'].properties['cat_path'])

In [6]:
movieIds

,movieId,movieId_size
0,<NA>,0
1,50,495
2,100,443
3,181,439
4,258,412
...,...,...
1676,1678,1
1677,1679,1
1678,1680,1
1679,1681,1


The highest movie id in the train dataset is 1682 with movie ID 0 being left for movies not seen in the train set.

Let's create a mock embedding matrix. In a regular scenario, that is where our pretrained embeddings would go.

In [7]:
pretrained_movie_embs = np.random.random((1682, 64))

Let us now feed this into our model.

## Building the model

In [8]:
model = mm.DCNModel(
    train.schema,
    depth=2,
    deep_block=mm.MLPBlock([64, 32]),
    prediction_tasks=mm.BinaryClassificationTask(target_column),
    embedding_options=mm.EmbeddingOptions(
        embeddings_initializers={
            "movieId": mm.TensorInitializer(pretrained_movie_embs),
        }
    )
)

2022-06-20 21:53:42.628823: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


## Training

In [9]:
%%time
opt = tf.keras.optimizers.Adagrad(learning_rate=1e-1)
model.compile(optimizer=opt, run_eagerly=False, metrics=[tf.keras.metrics.AUC()])
model.fit(train, validation_data=valid, batch_size=1024, epochs=10)

Epoch 1/10
89/89 [==============================] - 3s 9ms/step - loss: 0.6258 - auc: 0.6985 - val_loss: 0.6295 - val_auc: 0.6875
Epoch 2/10
89/89 [==============================] - 1s 6ms/step - loss: 0.6152 - auc: 0.7145 - val_loss: 0.6282 - val_auc: 0.6892
Epoch 3/10
89/89 [==============================] - 1s 6ms/step - loss: 0.6129 - auc: 0.7176 - val_loss: 0.6318 - val_auc: 0.6913
Epoch 4/10
89/89 [==============================] - 1s 5ms/step - loss: 0.6070 - auc: 0.7248 - val_loss: 0.6232 - val_auc: 0.6959
Epoch 5/10
89/89 [==============================] - 0s 5ms/step - loss: 0.5961 - auc: 0.7375 - val_loss: 0.6170 - val_auc: 0.7055
Epoch 6/10
89/89 [==============================] - 1s 6ms/step - loss: 0.5831 - auc: 0.7531 - val_loss: 0.6062 - val_auc: 0.7149
Epoch 7/10
89/89 [==============================] - 1s 5ms/step - loss: 0.5736 - auc: 0.7638 - val_loss: 0.6041 - val_auc: 0.7230
Epoch 8/10
89/89 [==============================] - 1s 6ms/step - loss: 0.5683 - auc: 0.76

The model trains and we have utilized pretrained embeddings 